In [1]:
import pandas as pd
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog
import sys

In [29]:
root=tk.Tk()
root.withdraw()
file=filedialog.askopenfilename(title="Select Master Headcount file...")
df=pd.read_excel(file,sheet_name="All")

In [37]:
#text_file = open("Output.txt", "w")
check_df=df[['Classification','SAP ID','Last Name','First Name','E Mail','Headcount type','Employee Status','Level','Office','MF Hire Date','Hire Date','Deloitte Service Date','Employee Type','Employment %','% Funding Allocation','% Time Allocation','FTE((Emp % * Funding %)/10000)','Gender','PM Scope','MF Personnel ID','SAP Job Title','Counselor Name','Counselor E Mail','Counselor Level','Personnel Area','Department','Department Leader','Business Unit','Business Unit Leader','Division','Division Leader','Function','Service Area']]
with open("Output.txt", 'w') as text_file:
    for column in check_df:
        # Seperate exception columns
        if (column=='MF Personnel ID') or (column=='MF Hire Date') or (column=='Deloitte Service Date') or (column=='Counselor Name') or (column=='Counselor E Mail') or          (column=='Counselor Level'):
            # Check if MF Personnel ID is missing for DTTS personnel
            if (column=='MF Personnel ID'):
                dtts_df=check_df[(check_df['Personnel Area'].str.startswith('DTTS',na=False))]
                if dtts_df['MF Personnel ID'].isnull().any():
                    text_file.write('MF Personnel ID is missing for {0} DTTS Personnel\n'.format(dtts_df['MF Personnel ID'].isnull().sum()))
                del dtts_df
            # Check if the missing MF Hire Date is for Non DTTL personnel
            if (column=='MF Hire Date'):
                not_non_dttl_df=check_df[(check_df['Classification']!='Non-DTTL')]
                if not_non_dttl_df['MF Hire Date'].isnull().any():
                    text_file.write('MF Hire date is missing for {0} Personnel (Other than Non-DTTL)\n'.format(not_non_dttl_df['MF Hire Date'].isnull().sum()))
                del not_non_dttl_df
            # Check if the missing Service date is for Contractors or retired partners
            if (column=='Deloitte Service Date'):
                not_cntrctr_rtd_prtnrs=check_df[(check_df['Employee Type']!='Contractor')&(check_df['Employee Type']!='Retired Partner')]
                if not_cntrctr_rtd_prtnrs['Deloitte Service Date'].isnull().any():
                    text_file.write('Service date is missing for {0} Personnel (Other than contractors & retired partners)\n'.format(not_cntrctr_rtd_prtnrs['Deloitte                         Service Date'].isnull().sum()))
                del not_cntrctr_rtd_prtnrs
            #Check if the counselor information is missing for more than 2 people
            if (column=='Counselor Name'):
                if (check_df['Counselor Name'].isnull().sum())>2:
                    text_file.write('Please check for the missing Counselor Name\n')
            if (column=='Counselor E Mail'):
                if (check_df['Counselor E Mail'].isnull().sum())>2:
                    text_file.write('Please check for the missing Counselor E-mail\n')
            if (column=='Counselor Level'):
                if (check_df['Counselor Level'].isnull().sum())>2:
                    text_file.write('Please check for the missing Counselor Level\n')
        # Check for the Null values in the mentioned columns.
        else:
            if check_df[column].isnull().any():
                text_file.write('{0}: {1} null values\n'.format(column,check_df[column].isnull().sum()))
    # Check for duplicates
    if check_df['SAP ID'].duplicated().sum()>0:
        text_file.write('Please check for duplicate values in SAP ID\n')
    if check_df['E Mail'].duplicated().sum()>0:
        text_file.write('Please check for duplicate values in Email\n')
    del check_df
    # Check for ethnicity- country
    ethnic_df=df[['Country','Ethnicity']]
    ethnic_df=ethnic_df[(ethnic_df['Ethnicity'].notnull())]
    if ethnic_df['Country'].nunique()>1:
        text_file.write('Ethincity information present in multiple countries\n')
    del ethnic_df
    # Check for contractor level, PM-scope and Compensation Guidelines
    level_emp=df[['Employee Type','Level','PM Scope','Compensation Guidelines','Employment %']]
    cont_emp=level_emp[level_emp['Employee Type']=='Contractor']
    cont_emp=cont_emp[cont_emp['Level']!=4]
    if cont_emp['Level'].nunique()>0:
        text_file.write("Contractor level is other than '4' \n")
    del cont_emp
    cont_emp=level_emp[level_emp['Employee Type']=='Contractor']
    cont_emp=cont_emp[cont_emp['PM Scope']!='Not in PM Scope']
    if cont_emp['PM Scope'].nunique()>0:
        text_file.write("Contractor PM Scope is other than 'Not in PM Scope' \n")
    del cont_emp
    cont_emp=level_emp[level_emp['Employee Type']=='Contractor']
    cont_emp=cont_emp[cont_emp['Compensation Guidelines']!='Not in Comp Scope']
    if cont_emp['Compensation Guidelines'].nunique()>0:
        text_file.write("Contractor Compensation Guidelines is other than 'Not in Comp Scope' \n")
    del cont_emp
    cont_emp=level_emp[level_emp['Employee Type']=='Contractor']
    cont_emp=cont_emp[cont_emp['Employment %']!=100]
    if cont_emp['Employment %'].nunique()>0:
        text_file.write("Contractor Employee% is other than 100% \n")
    del cont_emp
    #Check for Retired Partner level, PM-scope and Compensation Guidelines
    retd_part=level_emp[level_emp['Employee Type']=='Retired Partner']
    retd_part=retd_part[retd_part['Level']!=9]
    if retd_part['Level'].nunique()>0:
        text_file.write("Retired Partner level is other than '9' \n")
    del retd_part
    retd_part=level_emp[level_emp['Employee Type']=='Retired Partner']
    retd_part=retd_part[retd_part['PM Scope']!='Not in PM Scope']
    if retd_part['PM Scope'].nunique()>0:
        text_file.write("Retired Partner PM Scope is other than 'Not in PM Scope' \n")
    del retd_part
    retd_part=level_emp[level_emp['Employee Type']=='Retired Partner']
    retd_part=retd_part[retd_part['Compensation Guidelines']!='Not in Comp Scope']
    if retd_part['Compensation Guidelines'].nunique()>0:
        text_file.write("Retired Partner Compensation Guidelines is other than 'Not in Comp Scope.' \n")
    del retd_part
    retd_part=level_emp[level_emp['Employee Type']=='Retired Partner']
    retd_part=retd_part[retd_part['Employment %']!=100]
    if retd_part['Employment %'].nunique()>0:
        text_file.write("Retired Partner Employee% is other than 100%.' \n")
    del retd_part
    del level_emp
    # Level and SAP Job title No. Comparison.
    sap_job_df=df[['Level','SAP Job Title']]
    sap_job_df['Last']=sap_job_df['SAP Job Title'].str[-1]
    sap_job_df=sap_job_df[sap_job_df['Last'].apply(lambda x: x.isnumeric())]
    sap_job_df['Last']=sap_job_df['Last'].astype('int64')
    sap_job_df['Check']=sap_job_df.Level!=sap_job_df.Last
    sap_job_info=sap_job_df.Check.sum()
    if sap_job_info>0:
        text_file.write('SAP Job Title Number is not equal to Level for {0} Practitioner(s)\n'.format(sap_job_info))
    del sap_job_df
    del sap_job_info
    # Headcount Type- Employee Type mismatch:
    hctype_df=df[['Headcount type','Employee Type']]
    flaglist=[]
    for i in range(len(hctype_df)):
        if (hctype_df['Headcount type'][i]=='Reported Resources') and (hctype_df['Employee Type'][i] in ['Permanent','Secondment','Service Provider','Assignment',       'Temporary Employee','Cross Border Secondment']):
            flaglist.append(1)
        elif (hctype_df['Headcount type'][i]=='Other Resources') and (hctype_df['Employee Type'][i] in ['Contractor','Retired Partner','Non-DTTL Service Provider']):
            flaglist.append(1)
        elif (hctype_df['Headcount type'][i]=='Other Resources Non-DTTL') and (hctype_df['Employee Type'][i] in ['Non-DTTL Documentation','Non-DTTL Counselor']):
            flaglist.append(1)
        else:
            flaglist.append(0)
    hctype_df["flag"]=flaglist
    if (len(hctype_df['flag'])-hctype_df.flag.sum())!=0:
        text_file.write("Headcount Type - Employee Type Mismatch for {0} Practitioner(s) \n".format(len(hctype_df['flag'])-hctype_df.flag.sum()))
    del flaglist
    del hctype_df
    del i
    # Employee Type End Data check for Contractor, Secondment, Cross Border Secondment and Interns
    emptype_df=df[['Employee Type','Employee Type End Date']]
    emptype_df=emptype_df[emptype_df['Employee Type'].isin(['Contractor','Secondment','Cross Border Secondment','Intern','Interns'])]
    if emptype_df['Employee Type End Date'].isnull().sum()>0:
        text_file.write("Employee Type End Date missing for {0} Practitioner(s). \n".format(emptype_df['Employee Type End Date'].isnull().sum()))
    del emptype_df
    # Check if End Date Present for India - Assignment
    ind_assign_df=df[['Country','Employee Type','Employee Type End Date']]
    ind_assign_df=ind_assign_df[(ind_assign_df['Country']=='INDIA')&(ind_assign_df['Employee Type']=='Assignment')]
    if ind_assign_df['Employee Type End Date'].isnull().sum()-len(ind_assign_df)!=0:
        text_file.write("Employee Type End Date  present for INDIA - Assignment. \n")
    del ind_assign_df
    # recieving firm vs country
    rec_country_df=df[['Country','Receiving Firm']]
    rec_country_df.dropna(inplace=True)
    rec_country_df.reset_index(drop=True,inplace=True)
    flaglist=[]
    for i in range(len(rec_country_df)):
        if (rec_country_df['Receiving Firm'][i].upper()==rec_country_df['Country'][i].upper()):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='CARIBBEAN AND BERMUDA') and (rec_country_df['Country'][i].upper() in ['CAYMAN']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='CENTRAL EUROPE') and (rec_country_df['Country'][i].upper() in ['SLOVAKIA','CZECH REPUBLIC','BELGIUM']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='DTTS') and (rec_country_df['Country'][i].upper() in ['UNITED STATES']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='HONG KONG / CHINA') and (rec_country_df['Country'][i].upper() in ['CHINA']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='INDIA MF') and (rec_country_df['Country'][i].upper() in ['INDIA']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='LATCO') and (rec_country_df['Country'][i].upper() in ['COLOMBIA','VENEZUELA','ARGENTINA']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='SOUTHERN AFRICA') and (rec_country_df['Country'][i].upper() in ['SOUTH AFRICA']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='TRINIDAD AND TOBAGO') and (rec_country_df['Country'][i].upper() in ['REPUBLIC OF TRINIDAD AND TOBAGO']):
            flaglist.append(1)
        elif (rec_country_df['Receiving Firm'][i].upper()=='USI') and (rec_country_df['Country'][i].upper() in ['INDIA']):
            flaglist.append(1)
        else:
            flaglist.append(0)
    rec_country_df["flag"]=flaglist
    if (len(rec_country_df['flag'])-rec_country_df.flag.sum())!=0:
        text_file.write("Country - Recieving Firm mismatch for {0} Practitioner(s). \n".format(len(rec_country_df['flag'])-rec_country_df.flag.sum()))
    del flaglist
    del rec_country_df
    del i
    #DTTS professionals Country check + Reported resources DOB check + Managing director level
    dtts_df=df[['Headcount type','Personnel Area','Country','Date of Birth(MM/DD)','Level Name','Managing Director Level']]
    dtts_df=dtts_df[(dtts_df['Personnel Area'].str.startswith('DTTS',na=False))]
    if dtts_df['Country'].nunique()>1:
        text_file.write("Multiple countries showing in DTTS Professionals Country.\n")
    dtts_dob_df=dtts_df[(dtts_df['Headcount type']=='Reported Resources')]
    if dtts_dob_df['Date of Birth(MM/DD)'].isnull().sum()>0:
        text_file.write("Date of Birth Missing for {0} DTTS - Reported resource(s). \n".format(dtts_dob_df['Date of Birth(MM/DD)'].isnull().sum()))
    del dtts_dob_df
    dtts_df=dtts_df[(dtts_df['Level Name']=='Managing Director')]
    if dtts_df['Managing Director Level'].isnull().sum()>0:
        text_file.write("Managing Director level missing for {0} DTTS - Managing Director(s). \n".format(dtts_df['Managing Director Level'].isnull().sum()))
    del dtts_df
    # SAP Cost Center check for Non-DDTL resorces
    non_dttl_df=df[['Classification','SAP Cost Center']]
    non_dttl_df=non_dttl_df[non_dttl_df['Classification']!='Non-DTTL']
    if (non_dttl_df['SAP Cost Center'].values==55845).sum()>0:
        text_file.write("Non-DTTL Cost Center 55845 appearing for  {0} practitioner(s). \n".format((non_dttl_df['SAP Cost Center'].values==55845).sum()))
    del non_dttl_df
    # Level Vs Counselor level
    couns_df=df[['SAP ID','E Mail','Headcount type','Level','Counselor E Mail','Counselor Level']]
    email_df=df[['E Mail']]
    couns_df['Counselor Level']=couns_df['Counselor Level'].fillna(0).astype('int64')
    flaglist=[]
    flaglist2=[]
    for i in range(len(couns_df)):
            if (couns_df['Counselor Level'][i]>=couns_df['Level'][i]):
                flaglist.append(0)
            # We can add exception SAP IDs Here to remove them from the list.
            elif ((couns_df['Counselor Level'][i]<couns_df['Level'][i]) & (couns_df['SAP ID'][i] in [27105,2964,24080])):
                flaglist.append(0)
            else:
                flaglist.append(1)
            if(couns_df['E Mail'][i]==couns_df['Counselor E Mail'][i]):
                flaglist2.append(1)
            else:
                flaglist2.append(0)
    couns_df["flag"]=flaglist
    couns_df['flag2']=flaglist2
    if couns_df['flag'].sum()>0:
        text_file.write("There are {0} records where practitioner level is greater than Counselor Level. \n".format(couns_df['flag'].sum()))
    if(~couns_df['Counselor E Mail'].isin(email_df['E Mail'])).sum()>2:
        text_file.write("There are {0} counsellors, that are withdrawn. \n".format((~couns_df['Counselor E Mail'].isin(email_df['E Mail'])).sum()-2))
    if couns_df['flag2'].sum()>0:
        text_file.write("There are {0} records where practitioner Email is equal to Counselor Email. \n".format(couns_df['flag2'].sum()))
    del email_df
    del flaglist
    del flaglist2
    del couns_df
    del i
    #PM Scope Checks
    pmscope_df=df[['Headcount type','Level','PM Scope']]
    pmscope_df=pmscope_df[pmscope_df['Headcount type']=='Reported Resources']
    irpm_df=pmscope_df[pmscope_df['PM Scope']=='iRPM']
    if irpm_df['Level'].max() >7:
        text_file.write("Levels Greater than 7 are showing under iRPM. Please Check! \n")
    sfd_df=pmscope_df[pmscope_df['PM Scope']=='SF / Director']
    if sfd_df['Level'].nunique()>1:
        text_file.write(" SF / Director PM Scope has Multiple levels other than Level8. Please Check! \n")
    sfp_df=pmscope_df[pmscope_df['PM Scope']=='SF / Partner']
    if sfp_df['Level'].nunique()>2:
        text_file.write(" SF / Partner PM Scope has Multiple levels other than Level8 & Level9. Please Check! \n")
    del pmscope_df
    del irpm_df
    del sfd_df
    del sfp_df


